In [1]:
from google.colab import files
import pandas as pd

# Upload file
uploaded = files.upload()




Saving dataset (1).csv to dataset (1).csv


In [ ]:
# Load and clean
df = pd.read_csv("dataset (1).csv")  # Replace with your filename
traffic_map = {"low": 0.2, "average": 0.5, "high": 0.9}
df["TrafficConditionNearby"] = df["TrafficConditionNearby"].map(traffic_map)

# Optional: smaller dataset for speed
df_small = df.head(100)
df_small.to_csv("stream_data_clean.csv", index=False)

In [ ]:
# Encode vehicle types
vehicle_map = {'car': 1, 'bike': 0.5, 'truck': 1.5}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_map)

# Normalize occupancy and queue length
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df['QueueLengthNorm'] = (df['QueueLength'] - df['QueueLength'].min()) / (df['QueueLength'].max() - df['QueueLength'].min())

# Ensure numeric types
columns_to_fix = ['OccupancyRate', 'QueueLengthNorm', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleTypeWeight']
for col in columns_to_fix:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.dropna(subset=columns_to_fix)

In [ ]:
def linear_price_model(df, base_price=10, alpha=2.0):
    prices = []
    current_price = base_price
    for occ_rate in df['OccupancyRate']:
        next_price = current_price + alpha * occ_rate
        prices.append(next_price)
        current_price = next_price
    return prices

df['Price_Model_1'] = linear_price_model(df)

In [ ]:
def compute_demand(df, alpha=1, beta=1, gamma=1, delta=1, epsilon=1):
    demand = (
        alpha * df['OccupancyRate'] +
        beta * df['QueueLengthNorm'] -
        gamma * df['TrafficConditionNearby'] +
        delta * df['IsSpecialDay'] +
        epsilon * df['VehicleTypeWeight']
    )
    return (demand - demand.min()) / (demand.max() - demand.min())

df['Demand'] = compute_demand(df)

def dynamic_price(df, base_price=10, lam=0.5):
    return base_price * (1 + lam * df['Demand']).clip(lower=0.5, upper=2.0)

df['Price_Model_2'] = dynamic_price(df)

In [ ]:
from math import radians, cos, sin, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

def apply_competition_adjustment(df):
    adjusted_prices = []
    for i, row in df.iterrows():
        nearby = df[
            (df['LocationID'] != row['LocationID']) &
            (df.apply(lambda x: haversine(row['Latitude'], row['Longitude'], x['Latitude'], x['Longitude']), axis=1) < 0.5)
        ]
        min_competitor_price = nearby['Price_Model_2'].min() if not nearby.empty else row['Price_Model_2']
        adjusted_price = min(row['Price_Model_2'], min_competitor_price + 1)
        adjusted_prices.append(adjusted_price)
    return adjusted_prices

df['Price_Model_3'] = apply_competition_adjustment(df)

In [ ]:
def price_volatility(prices):
    return np.mean(np.abs(np.diff(prices)))

def utilization_score(df):
    return np.mean(np.abs(df['OccupancyRate'] - 0.8))

def revenue(df, price_col):
    return (df[price_col] * df['Occupancy']).sum()

print("Model 1 Volatility:", price_volatility(df['Price_Model_1']))
print("Model 2 Volatility:", price_volatility(df['Price_Model_2']))
print("Model 3 Volatility:", price_volatility(df['Price_Model_3']))

print("Model 1 Utilization Score:", utilization_score(df))
print("Model 2 Utilization Score:", utilization_score(df))
print("Model 3 Utilization Score:", utilization_score(df))

print("Model 1 Revenue:", revenue(df, 'Price_Model_1'))
print("Model 2 Revenue:", revenue(df, 'Price_Model_2'))
print("Model 3 Revenue:", revenue(df, 'Price_Model_3'))

In [ ]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

import numpy as np
x = np.arange(len(df))

p = figure(title="Pricing Over Time", x_axis_label="Time", y_axis_label="Price")
p.line(x, df['Price_Model_1'], legend_label="Model 1", color="blue")
p.line(x, df['Price_Model_2'], legend_label="Model 2", color="green")
p.line(x, df['Price_Model_3'], legend_label="Model 3", color="red")
p.legend.location = "top_left"
show(p)

In [ ]:
import pathway as pw

class ParkingSchema(pw.Schema):
    ID: str
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: float
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str

stream = pw.io.csv.read("stream_data_clean.csv", schema=ParkingSchema, mode="static")


In [ ]:
def vehicle_weight(vehicle: str) -> float:
    if vehicle == "car":
        return 1.0
    elif vehicle == "bike":
        return 0.5
    elif vehicle == "truck":
        return 1.5
    else:
        return 1.0

stream += stream.select(
    OccupancyRate = stream.Occupancy / stream.Capacity,
    VehicleTypeWeight = pw.apply(vehicle_weight, stream.VehicleType)
)

stream += stream.select(
    Demand = (
        1.0 * stream.Occupancy / stream.Capacity +
        1.0 * (stream.QueueLength / 10) -
        1.0 * stream.TrafficConditionNearby +
        1.0 * stream.IsSpecialDay +
        1.0 * stream.VehicleTypeWeight
    )
)

def normalize_demand(d: float) -> float:
    return min(1.0, max(0.0, (d - 1) / 10))

stream += stream.select(
    NormDemand = pw.apply(normalize_demand, stream.Demand)
)

BASE_PRICE = 10
LAMBDA = 0.5

stream += stream.select(
    FinalPrice = BASE_PRICE * (1 + LAMBDA * stream.NormDemand)
)

pw.io.jsonlines.write(
    stream.select(
        stream.ID,
        stream.Occupancy,
        stream.Demand,
        stream.FinalPrice
    ),
    "stream_output.jsonl"
)

pw.run()


In [ ]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import Category10
from bokeh.models import Legend

output_notebook()

# Load output
df_output = pd.read_json("stream_output.jsonl", lines=True)

top_ids = df_output['ID'].value_counts().head(5).index.tolist()

p = figure(title="Real-Time Parking Prices", x_axis_label="Entry Index", y_axis_label="FinalPrice")
legend_items = []

for idx, lot_id in enumerate(top_ids):
    df_lot = df_output[df_output["ID"] == lot_id].reset_index()
    line = p.line(df_lot.index, df_lot["FinalPrice"], color=Category10[10][idx], line_width=2)
    legend_items.append((f"Lot {lot_id}", [line]))

p.legend.items = legend_items
p.legend.location = "top_left"
show(p)
